In [1]:
import sys
sys.path.append('../src')

import importlib
import numpy as np
from sklearn.covariance import empirical_covariance

import policies 

# Covariance Constrained Policy Testing

Testing the covariance constrained policy. Initially, this was giving a lot of violations. Initially thought that the problem might just not be feasible but after inspection it was. I switched the solver being used by cvxpy to Gurobi, which resolved the issue. 

The initial line in this, %%capture, redirects stdout from printing out the output. This is much nicer than redirecting the stdout manually, which doesn't work as anticipated inside of jupyter notebooks. Note: if your code is multithreaded, this no longer works for some reason. 

In [6]:
%%capture

importlib.reload(policies)

n = 50
n_features = 6
pred_dim = 6

np.random.seed(30)

# Gaussian xs, with true labels having a different linear relationship with xs in each coordinate
xs = np.random.normal(size=(n, n_features))
slopes = np.random.uniform(size = n_features)
ys = np.multiply(xs, slopes)

def meta_model(coord, slopes):
    def model(xs):
        preds = np.random.normal(size=(n, pred_dim))
        true_ys = np.multiply(xs, slopes)
        preds[:,coord] = true_ys[:,coord]
        return preds
    return model

model = meta_model(0, slopes)

alpha = 0.5
policy = policies.VarianceConstrained(pred_dim, model, 0.1, alpha, ys)
out = policy.run_given_preds(ys);

In [7]:
out

array([[1.00002772e-10, 1.01898587e-07, 7.38446508e-09, 9.99999815e-01,
        3.53529912e-08, 4.01164393e-08],
       [2.96881002e-11, 3.00033367e-01, 5.42156947e-08, 1.19107103e-09,
        6.99966253e-01, 3.24739202e-07],
       [1.85415371e-10, 1.34218179e-08, 8.82189287e-08, 9.99999848e-01,
        7.75098790e-09, 4.27674899e-08],
       [2.12656567e-11, 6.87291825e-08, 8.73373638e-09, 3.82770249e-10,
        2.80138361e-11, 9.99999922e-01],
       [6.43062758e-11, 9.84896418e-09, 2.99122905e-01, 1.05979690e-09,
        7.00877082e-01, 2.14731828e-09],
       [3.03638175e-12, 1.36254688e-09, 9.99999998e-01, 2.71606920e-11,
        6.38004950e-12, 8.79946302e-10],
       [3.53786621e-01, 1.25668874e-11, 1.27909305e-11, 6.18090997e-11,
        6.46213379e-01, 1.29075415e-11],
       [4.34172094e-08, 2.72069579e-08, 1.79701650e-08, 2.98305972e-01,
        7.01693918e-01, 2.13061344e-08],
       [8.94744143e-10, 4.75937405e-08, 5.46548664e-08, 2.98306681e-01,
        7.01693176e-01, 

Sanity check: 

1. Does each row sum to 1? Yes! Or at least, up to a tolerance, since there will be small floating point errors.

In [8]:
tolerance = 1e-3

print(np.sum(out, axis=1))
print("Violations: ", n-sum((np.isclose(np.sum(out, axis=1), np.ones(50), atol=tolerance))))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1.]
Violations:  0


2. Is each constraint bounded by 0 and 1?

In [9]:
print(f"Number of allocations greater than 1: {np.sum(out - tolerance > 1)}")
print(f"Number of allocations less than 0: {np.sum(out+tolerance < 0)}")

Number of allocations greater than 1: 0
Number of allocations less than 0: 0


3. Are the variance conditions being approx satisfied? I wrote yes here before but it seems like no?

In [10]:
cov = empirical_covariance(ys, assume_centered=False)

viol = np.zeros(len(out))
for i in range(len(out)):
    viol[i] = np.matmul(np.matmul(out[i], cov), np.transpose(out[i]))

print(f"Max violation: {alpha}")
print(f"Number of variance constraints which violate the max allowed variance: {sum(viol > alpha+0.1)}")

Max violation: 0.5
Number of variance constraints which violate the max allowed variance: 0


3. Is the covariance matrix actually measuring the correct thing? A sanity check that the diagonal of the matrix is equal to empirical variance.


In [11]:
print(f"Variances of each coordinate of the ys: \n {np.var(ys, axis=0)}")

print(f"Diagonal of the covariance matrix: \n {np.diagonal(cov)}")

Variances of each coordinate of the ys: 
 [0.82798883 0.00104122 0.00179195 0.10912913 1.02202326 0.00313415]
Diagonal of the covariance matrix: 
 [0.82798883 0.00104122 0.00179195 0.10912913 1.02202326 0.00313415]


4. Are all the constraints convex and are things feasible?

If C is positive semi-definite, then xCx^T <= val is a convex constraint on R^n. Maybe something went wrong in calculation of C? 

If the eigenvalues of C are positive, then C must be positive semi-definite. Here we see this is the case:

In [30]:
np.linalg.eigvals(cov) > 0

array([ True,  True,  True,  True,  True,  True])

If the only constraint was covariance, xCx^T will be bounded by the eigenvalues of covariance matrix, so this is a good spot check for how feasible things are

In [31]:
np.linalg.eigvals(cov)

array([1.04541420e+00, 8.10321387e-01, 1.04229047e-01, 2.73097313e-03,
       8.41962782e-04, 1.57097101e-03])

# Linear Constraint Policy Testing

In [ ]:
%%capture

importlib.reload(policies)

n = 50
n_features = 6
pred_dim = 6

np.random.seed(30)

# Gaussian xs, with true labels having a different linear relationship with xs in each coordinate
xs = np.random.normal(size=(n, n_features))
slopes = np.random.uniform(size = n_features)
ys = np.multiply(xs, slopes)

def meta_model(coord, slopes):
    def model(xs):
        preds = np.random.normal(size=(n, pred_dim))
        true_ys = np.multiply(xs, slopes)
        preds[:,coord] = true_ys[:,coord]
        return preds
    return model

model = meta_model(0, slopes)

alpha = 0.5
policy = policies.LinearConstraint(pred_dim, model, 0.1, alpha, ys)
out = policy.run_given_preds(ys);